## English to Hindi Translation - Using Encoder Decoder Model

Dataset Link :
https://www.kaggle.com/datasets/aiswaryaramachandran/hindienglish-corpora

In [ ]:
import pandas as pd
import numpy as np
import os
from string import digits
import matplotlib.pyplot as plt
import re
import string
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input,Dense,LSTM,Embedding
from keras.models import Model

## Load Dataset

In [ ]:
!pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!kaggle datasets download -d aiswaryaramachandran/hindienglish-corpora

 36% 5.00M/13.9M [00:00<00:00, 43.5MB/s]
100% 13.9M/13.9M [00:00<00:00, 68.7MB/s]


In [ ]:
!unzip /content/hindienglish-corpora.zip

Archive:  /content/hindienglish-corpora.zip
  inflating: Hindi_English_Truncated_Corpus.csv  


In [ ]:
df = pd.read_csv("/content/Hindi_English_Truncated_Corpus.csv")

In [ ]:
df.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [ ]:
df['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [ ]:
data = df[df['source']=='ted']

In [ ]:
data.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है


In [ ]:
data.shape

(39881, 3)

In [ ]:
pd.isnull(data).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [ ]:
data = data[~pd.isnull(data['english_sentence'])]
data.shape

(39881, 3)

In [ ]:
data.duplicated().sum()

1078

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
# Let's pic any 25000 rows

data = data.sample(n=25000,random_state=42)
data.shape

(25000, 3)

In [ ]:
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.lower())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.lower())

In [ ]:
# remove quotes
data['english_sentence'] = data['english_sentence'].apply(lambda x: re.sub("'",'',x))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub("'",'',x))

In [ ]:
# Remove special character
exclude = set(string.punctuation)  # all special character
data['english_sentence'] = data['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# Remove all number from text
remove_digits = str.maketrans('','',digits)
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.translate(remove_digits))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
# Remove Extra Space
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.strip())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.strip())
data['english_sentence'] = data['english_sentence'].apply(lambda x: re.sub(" +"," ",x))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub(" +"," ",x))

In [ ]:
# Add start and end token
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: 'START_ '+ x + ' _END')

In [ ]:
data.head()

,source,english_sentence,hindi_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...


In [ ]:
# Get English and Hindi Vocabulary

all_eng_words = set()

for eng in data['english_sentence']:
  for word in eng.split():
    if word not in all_eng_words:
      all_eng_words.add(word)

all_hin_words = set()

for hin in data['hindi_sentence']:
  for word in hin.split():
    if word not in all_hin_words:
      all_hin_words.add(word)

In [ ]:
len(all_eng_words)

14030

In [ ]:
len(all_hin_words)

17540

In [ ]:
# add length of sentence in your dataset
data['length_english_sentence'] = data['english_sentence'].apply(lambda x:len(x.split(" ")))
data['length_hindi_sentence'] = data['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [ ]:
data.head()

,source,english_sentence,hindi_sentence,length_english_sentence,length_hindi_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...,11,16
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END,2,5
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END,7,8
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END,4,6
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...,16,20


In [ ]:
data[data['length_english_sentence']>30].shape

(0, 5)

In [ ]:
data = data[data['length_english_sentence']<=20]
data = data[data['length_hindi_sentence']<=20]

In [ ]:
data.shape

(24774, 5)

In [ ]:
print('MAximum length of Hindi Sentence',max(data['length_hindi_sentence']))
print('MAximum length of English Sentence',max(data['length_english_sentence']))

MAximum length of Hindi Sentence 20
MAximum length of English Sentence 20


In [ ]:
max_length_src = max(data['length_hindi_sentence'])
max_length_tar = max(data['length_english_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hin_words))
num_encoder_token = len(all_eng_words)
num_decoder_token = len(all_hin_words)
num_encoder_token,num_decoder_token

(14030, 17540)

In [ ]:
num_decoder_token += 1 #for zero padding"

In [ ]:
input_token_index = dict([(word,i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word,i+1) for i, word in enumerate(target_words)])

In [ ]:
rev_input_token_index = dict([(i,word) for word,i in input_token_index.items()])
rev_target_token_index = dict([(i,word) for word,i in target_token_index.items()])

In [ ]:
data = shuffle(data)
data.head(10)

,source,english_sentence,hindi_sentence,length_english_sentence,length_hindi_sentence
66645,ted,of course you got it wrong,START_ आप बिलकुल ही गलत सोच रहे हो _END,6,9
25987,ted,is the dharavi slum of mumbai,START_ धारावी झुग्गी का इलाका मुंबई में _END,6,8
116627,ted,one when those teachers are doing that,START_ एक तो ये कि जब ये टीचर ऐसा करते हैं _END,7,12
16347,ted,i didnt have have many personal friends,START_ मेरा कोई मित्र भी नहीं था _END,7,8
17870,ted,the un wanted to rush people and supplies to t...,START_ संयुक्त राष्ट्र क्षेत्र में लोगों को और...,11,15
126559,ted,one with the creator,START_ परम पिता में _END,4,5
68506,ted,by country in february of just one year ago,START_ देशों के आधार पर फ़रवरी में केवल एक साल...,9,12
14867,ted,im here to talk to you about how globalized we...,START_ भूमंडलीकृत कैसे हम कर रहे हैं _END,11,8
95228,ted,this is a training ground,START_ यह एक प्रशिक्षण जमीन है _END,5,7
17369,ted,for the power of cartoons,START_ कार्टूनों की शक्ति की _END,5,6


In [ ]:
# split data into train test split
X,y = data['english_sentence'] , data['hindi_sentence']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=31)
X_train.shape,X_test.shape

((19819,), (4955,))

In [ ]:
## Save data
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_token),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

## Encoder-Decoder Archietecture

In [ ]:
latent_dim = 300

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_token,latent_dim,mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim,return_state=True)
encoder_outputs , state_h ,state_c = encoder_lstm(enc_emb)
encoder_states = [state_h,state_c]

In [ ]:
# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_token,latent_dim,mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs , _ ,_ = decoder_lstm(dec_emb,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_token,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# define Model
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 300)            4209000   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 300)            5262300   ['input_2[0][0]']             
                                                                                              

In [ ]:
train_sample = len(X_train)
val_sample = len(X_test)
epochs = 100
batch_size=128

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
model.fit_generator(generator=generate_batch(X_train,y_train,batch_size),
                    steps_per_epoch=train_sample//batch_size,
                    epochs=100,
                    validation_data = generate_batch(X_test,y_test,batch_size=batch_size),
                    validation_steps = val_sample//batch_size

                    )

<ipython-input-56-081c95ad281a>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=generate_batch(X_train,y_train,batch_size),


Epoch 1/100
154/154 [==============================] - 53s 345ms/step - loss: 4.1681 - val_loss: 5.2102
Epoch 2/100
154/154 [==============================] - 52s 338ms/step - loss: 4.1391 - val_loss: 5.2187
Epoch 3/100
154/154 [==============================] - 53s 342ms/step - loss: 4.1092 - val_loss: 5.2047
Epoch 4/100
154/154 [==============================] - 62s 404ms/step - loss: 4.0797 - val_loss: 5.2079
Epoch 5/100
154/154 [==============================] - 60s 392ms/step - loss: 4.0501 - val_loss: 5.2101
Epoch 6/100
154/154 [==============================] - 61s 395ms/step - loss: 4.0188 - val_loss: 5.2133
Epoch 7/100
154/154 [==============================] - 52s 335ms/step - loss: 3.9909 - val_loss: 5.2136
Epoch 8/100
154/154 [==============================] - 59s 384ms/step - loss: 3.9603 - val_loss: 5.2269
Epoch 9/100
154/154 [==============================] - 60s 390ms/step - loss: 3.9299 - val_loss: 5.2274
Epoch 10/100
154/154 [==============================] - 59s 384m

In [ ]:
model.save_weights('nmt_weights_100.h5')

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = rev_target_token_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 18ms/step
Input English sentence: of dollars a year in business
Actual Hindi Translation:  करोंड़ों डॉलर की व्यापार के। 
Predicted Hindi Translation:  करीब लाख लोगो के मुकाबले 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-2])

1/1 [==============================] - 0s 19ms/step
Input English sentence: can be if were given the opportunity to have a voice
Actual Hindi Translation:  यदि हमें आवाज उठाने का और एक दुसरे से बांटने का मौका 
Predicted Hindi Translation:  की हमें उस गाँव में आते हैं जो एक बड़ा समाज में नह
